In [253]:
# move this down
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
ds = pd.read_csv('scotch_review.csv')

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pickle
import csv

In [3]:
URL0 = 'https://flaviar.com/bottles/scotch'

page0 = requests.get(URL0)

soup0 = BeautifulSoup(page0.text, 'lxml')

In [173]:
# scraps website for bottle edition

bottle_names = []
bottle_editions = []
bottle_types = []



class0 = soup0.find('div', {'id': 'content', 'class': 'overflow-hidden'})
class1 = class0.find('div', {'class': 'fullWidth pattern-bg sand-bg-mobile padding-top-50 padding-top-mobile-40 padding-bottom-50'})
class2 = class1.find('div', {'class': 'container'})
class3 = class2.find('div', {'class': 'row'})
class4 = class3.find('div', {'class': 'col-md-12'})
class5 = class4.find('div', {'id': 'previousboxes', 'class': 'row margin-left-0 margin-right-0 padding-left-mobile-10 padding-right-mobile-10 bottles'})
class6 = class5.find('div', {'class': 'row'})
class7 = class6.find_all('div', {'class': 'col-lg-3 col-md-4 col-sm-6'})


for i in class7:
    class8 = i.find('a', {'class': 'single-item bottle'})    
    class9 = class8.find('div', {'class': 'holder'})
    class10 = class9.find('div', {'class': 'about flex flex--space-between'})
    class11 = class10.find('div', {'class': 'desc'})
    
    bottle_name = class11.find('div', {'class': 'name'})
    bottle_edition = class11.find('div', {'class': 'edition'})
    bottle_type = class11.find('div', {'class': 'type'})
    
    bottle_names.append(bottle_name.text.replace("'","").replace('the ','').replace(' ','-'))
    bottle_editions.append(bottle_edition.text.replace("'","").replace(' ','-').replace('No.', 'no'))
    bottle_types.append(bottle_type.text)
    



In [174]:
# Creates url for all editions in selected bottle type(s)
# Type choosen when URL0 is defined

url_editions = []
BOTTLE_COUNT = len(bottle_editions)
for n in range(BOTTLE_COUNT):
    url_editions.append((str('https://flaviar.com/' + bottle_names[n] + '/' + bottle_editions[n])))
    

In [379]:
#print(url_editions[:4])

# Fixing urls

url_editions[6] = 'https://flaviar.com/Aberlour/Aberlour-18-Year-Old'
url_editions[12] = 'https://flaviar.com/glenlivet/the-glenlivet-18-year-old'
url_editions[13] = 'https://flaviar.com/bruichladdich/bruichladdich-scottish-barley-malt-whisky'
url_editions[17] = 'https://flaviar.com/glenrothes/glenrothes-25-year-old'
url_editions[31] = 'https://flaviar.com/port-askaig/port-askaig-110-proof'
url_editions[33] = 'https://flaviar.com/bunnahabhain/bunnahabhain-stiuireadair'
url_editions[37] = 'https://flaviar.com/aberlour/aberlour-16-year-old-40'
url_editions[38] = 'https://flaviar.com/port-askaig/port-askaig-8-year-old-scotch-whisky'
url_editions[40] = 'https://flaviar.com/alexander-murray/alexander-murray-monumental-blend-18-year-old'
url_editions[42] = 'https://flaviar.com/compass-box/compass-box-the-story-of-the-spaniard'
url_editions[48] = 'https://flaviar.com/glenrothes/glenrothes-bourbon-cask-reserve-speyside'
url_editions[54] = 'https://flaviar.com/balvenie/the-balvenie-21-year-old-port-wood-finish'
url_editions[66] = 'https://flaviar.com/glenmorangie/glenmorangie-allta-private-edition-no-10'
url_editions[67] = 'https://flaviar.com/glenmorangie/glenmorangie-spios-private-edition-single-malt'
url_editions[68] = 'https://flaviar.com/bunnahabhain/bunnahabhain-toiteach-a-dha'
url_editions[69] = 'https://flaviar.com/orphan-barrel/orphan-barrel-foragers-keep'
url_editions[79] = 'https://flaviar.com/glenallachie/glenallachie-12-year-old'
url_editions[84] = 'https://flaviar.com/compass-box/compass-box-no-name-no2'
url_editions[85] = 'https://flaviar.com/glenfiddich/glenfiddich-grand-cru'
url_editions[88] = 'https://flaviar.com/royal-brackla/royal-brackla-16-years-old'
url_editions[90] = 'https://flaviar.com/ardbeg/ardbeg-traighbhan-19-year-old-2019-edition-700ml'
url_editions[91] = 'https://flaviar.com/mortlach/mortlach-16-year-distillers-dram'
url_editions[92] = 'https://flaviar.com/mortlach/mortlach-12-year-the-wee-witchie'
url_editions[94] = 'https://flaviar.com/kilchoman/kilchoman-us-small-batch-limited-edition'
url_editions[96] = 'https://flaviar.com/compass-box/compass-box-myths-legends-i'
url_editions[97] = 'https://flaviar.com/compass-box/compass-box-myths-legends-iii'
url_editions[104] = 'https://flaviar.com/johnnie-walker/johnnie-walker-10-year-old-blended-scotch-whisky-the-jane-walker-edition-2'
url_editions[127] = 'https://flaviar.com/macnairs/macnairs-lum-reek-peated-blended-scotch-whisky'
url_editions[128] = 'https://flaviar.com/macnairs/macnairs-lum-reek-peated-blended-scotch-whisky'
url_editions[130] = 'https://flaviar.com/macallan/the-macallan-15-year-old-double-cask'
url_editions[132] = 'https://flaviar.com/bruichladdich/bruichladdich-black-art-81'
url_editions[135] = 'https://flaviar.com/bruichladdich/bruichladdich-port-charlotte-olc-01-2010'
url_editions[139] = 'https://flaviar.com/hotaling-co/daftmill-2006-summer-batch-release-scotch-whisky'
url_editions[140] = 'https://flaviar.com/glenfiddich/glenfiddich-fire-cane-experimental-series'
url_editions[162] = 'https://flaviar.com/dalmore/dalmore-18-year-old'
url_editions[169] = 'https://flaviar.com/bunnahabhain/bunnahabhain-scotch-single-malt-18-year'
url_editions[185] = 'https://flaviar.com/chivas-regal/chivas-regal-royal-salute-21-yo'
url_editions[198] = 'https://flaviar.com/glendronach/glendronach-12-year-old-original'
url_editions[203] = 'https://flaviar.com/tomatin/tomatin-12-year-old-2002-cuatro-2-manzanilla-sherry-cask-finish'
url_editions[204] = 'https://flaviar.com/longrow/longrow-single-malt-peated'
url_editions[207] = 'https://flaviar.com/sia/sia-scotch-whisky'
url_editions[208] = 'https://flaviar.com/oban/oban-little-bay-single-malt-scotch-whisky'
url_editions[211] = 'https://flaviar.com/glendronach/glendronach-scotch-single-malt-18-year-allardice'
url_editions[214] = 'https://flaviar.com/glenlivet/the-glenlivet-nadurra-oloroso'
url_editions[222] = 'https://flaviar.com/scallywag/scallywag'
url_editions[226] = 'https://flaviar.com/kilchoman/kilchoman-100-islay'
url_editions[233] = 'https://flaviar.com/glendronach/glendronach-21yo-parliament'
url_editions[234] = 'https://flaviar.com/laphroaig/laphroaig-scotch-single-malt-triple-wood'
url_editions[248] = 'https://flaviar.com/glenlivet/glenlivet-nadurra-peated-whisky-cask-finish'
url_editions[249] = 'https://flaviar.com/glenlivet/the-glenlivet-nadurra-first-fill-selection'
url_editions[260] = 'https://flaviar.com/dalmore/dalmore-25-year-old'
url_editions[266] = 'https://flaviar.com/johnnie-walker/johnnie-walker-double-black-whisky'
url_editions[277] = 'https://flaviar.com/dalmore/dalmore-king-alexander-iii'
url_editions[283] = 'https://flaviar.com/dalmore/dalmore-12-year-old-whisky'
url_editions[288] = 'https://flaviar.com/macallan/the-macallan-fine-oak-scotch-single-malt-17-year-old'

In [380]:


appearances = []
noses = []
palates = []
finishes = []


for url in range(288,len(url_editions)):
    
        
    URL1 = url_editions[url]
    
    page1 = requests.get(URL1)

    soup1 = BeautifulSoup(page1.text, 'lxml')
    
    
    current_url = []
    current_url.append(URL1)
    
    profile_class0 = soup1.find('div', {'id': 'content', 'class': 'overflow-hidden'})
    profile_class1 = profile_class0.find('div', {'id': 'info', 'class': 'container'})
    profile_class2 = profile_class1.find('div', {'class': 'row shrink flex flex--stretch margin-bottom-50 margin-bottom-mobile-0'})
    profile_class3 = profile_class2.find('div', {'class': 'col-md-8 col-sm-12 col-xs-12 element info-description flex _flex-2 fit2EdgeOnMobile'})
    profile_class4 = profile_class3.find('div', {'class': 'introduce pattern-bg equal moduleTabs collapsable-content-box'})
    profile_class5 = profile_class4.find('div', {'id': 'tasting-notes','class': 'body collapse in more-body-padding'})
    profile_class6 = profile_class5.find('div', {'class': 'p-content'})
    

          
    if(profile_class6.find('br')) is not None and URL1 != 'https://flaviar.com/aberlour/aberlour-16-year-old-40':
        appearances.append(str(profile_class6.find('br').next_sibling).strip('\r\n'))

        noses.append(str(profile_class6.find('br').next_sibling.next_sibling.next_sibling.next_sibling.
                next_sibling.next_sibling.next_sibling.next_sibling).strip('\r\n'))

        palates.append(str(profile_class6.find('br').next_sibling.next_sibling.next_sibling.next_sibling.
                next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                next_sibling.next_sibling.next_sibling.next_sibling).strip('\r\n'))

        finishes.append(str(profile_class6.find('br').next_sibling.next_sibling.next_sibling.next_sibling.
                next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                next_sibling.next_sibling.next_sibling.next_sibling).strip('\r\n'))
    else:
        
        for p in profile_class6.find_all('div'):


            if (p.text == '\nAppearance / Color'):

                appearances.append(str(profile_class6.div.next_sibling.next_sibling.text).strip('\r\n\t'))

            elif (p.text == '\nNose / Aroma / Smell'):
                noses.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                                 next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                 text).strip('\r\n\t'))

            elif (p.text == '\nFlavor / Taste / Palate'):
                palates.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                                   next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                   next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                   next_sibling.text).strip('\r\n\t')) 

            elif (p.text == '\nFinish'):
                finishes.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                                    next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                    next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                    next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                    next_sibling.next_sibling.text).strip('\r\n\t'))

    


In [381]:
print(len(url_editions))
print(len(set(url_editions)))
print(len(appearances))
print(current_url[-1])
print(url_editions[288])


289
288
1
https://flaviar.com/macallan/the-macallan-fine-oak-scotch-single-malt-17-year-old
https://flaviar.com/macallan/the-macallan-fine-oak-scotch-single-malt-17-year-old


In [47]:
with open('scotch_review.csv', 'w', newline = '') as f:
    for_to_write = csv.writer(f)
    
    for_to_write.writerow(['edition', 'appearance', 'nose', 'palate', 'finish'])

    for n in range(len(bottle_editions)):
            
        for_to_write.writerow([bottle_edition[n], appearances[n], noses[n], palates[n], finishes[n]])
        

KeyError: 0

In [49]:
with open('scotch_review.csv', 'w', newline = '') as f:
    field_names = ['edition', 'appearance', 'nose', 'palate', 'finish']
    for_to_write = csv.DictWriter(f, fieldnames = field_names)
    

    for n in range(len(bottle_editions)):
            
        for_to_write.writerow({'edition' : bottle_edition[n], 'appearance' : appearances[n],
                               'noses' : noses[n], 'palate' : palates[n], 'finish' : finishes[n]})
        

KeyError: 0

In [ ]:
pickle.dump(profile_class6.text, open( "tasting-notes.p", "wb" ))
#print(pickle.load(open( "tasting-notes.p", "rb" )))



pickle.dump(bottle_editions, open( "bottle-editions.p", "wb" ))
#print(pickle.load(open( "bottle-editions.p", "rb" )))